### Implementing a CNN for text classification in tensorflow
**Guide Link**: http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

**TODOS**: apply it to emotion classificaiton

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from helpers import *
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

/home/ellfae/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Data and Preprocessing

* Load positive and negative sentences from the raw data files.
* Clean the text data using the same code as the original paper.
* Pad each sentence to the maximum sentence length, which turns out to be 59. We append special <PAD> tokens to all other sentences to make them 59 words. Padding sentences to the same length is useful because it allows us to efficiently batch our data since each example in a batch must be of the same length.
* Build a vocabulary index and map each word to an integer between 0 and 18,765 (the vocabulary size). Each sentence becomes a vector of integers.

### Parameters
---

In [3]:
# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")

# these are both used to add regularization; it is needed where data is small in size
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [4]:
FLAGS = tf.flags.FLAGS

In [5]:
FLAGS._parse_flags()

['-f',
 '/run/user/1000/jupyter/kernel-890f3a35-458f-4167-b570-107a3a2e03f1.json']

In [6]:
print ("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=128
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos



### Data Preparation
---

In [7]:
# laod data (pos: 5331 neg: 5331)
print("loading data...")
x_text, y = cnn_data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

loading data...


In [8]:
y[:2]

array([[0, 1],
       [0, 1]])

In [9]:
x_text[:10]

["the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal",
 "the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words cannot adequately describe co writer director peter jackson 's expanded vision of j r r tolkien 's middle earth",
 'effective but too tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start',
 "emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one",
 'the film provides some great insight into the neurotic mindset of all comics even those who have reached the absolute top of the game',
 'offers that rare combination of entertainment and education',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions',
 "steers turns in a snappy screenplay that cu

#### Building vocbulary

In [10]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])

In [11]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)

In [12]:
x = np.array(list(vocab_processor.fit_transform(x_text)))

In [13]:
x.shape

(10662, 56)

#### Randomly shuffle data

In [14]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

#### Split train/test set

In [15]:
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))

In [16]:
dev_sample_index

-1066

In [17]:
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

In [ ]:
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 18758
Train/Dev split: 9596/1066


### Training
---

In [ ]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = cnn_model_helpers.TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = cnn_data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.


2017-11-04 19:04:07,185 : INFO : Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.


INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.


2017-11-04 19:04:07,199 : INFO : Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.


INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.


...

INFO:tensorflow:Summary name output/b:0/grad/hist is illegal; using output/b_0/grad/hist instead.


2017-11-04 19:04:07,277 : INFO : Summary name output/b:0/grad/hist is illegal; using output/b_0/grad/hist instead.


INFO:tensorflow:Summary name output/b:0/grad/sparsity is illegal; using output/b_0/grad/sparsity instead.


2017-11-04 19:04:07,284 : INFO : Summary name output/b:0/grad/sparsity is illegal; using output/b_0/grad/sparsity instead.


Writing to /home/ellfae/Dropbox/WORK/PhD Research/elvis_emotion_2017/runs/1509793447

2017-11-04T19:04:08.013092: step 1, loss 4.70679, acc 0.515625
2017-11-04T19:04:08.288562: step 2, loss 3.33291, acc 0.46875
2017-11-04T19:04:08.569549: step 3, loss 2.30769, acc 0.59375
2017-11-04T19:04:08.837223: step 4, loss 1.86117, acc 0.5
2017-11-04T19:04:09.192384: step 5, loss 1.93099, acc 0.5625
2017-11-04T19:04:09.481266: step 6, loss 2.00088, acc 0.5625
2017-11-04T19:04:09.740068: step 7, loss 2.60071, acc 0.5
2017-11-04T19:04:09.993488: step 8, loss 2.55949, acc 0.515625
2017-11-04T19:04:10.258245: step 9, loss 2.03948, acc 0.578125
2017-11-04T19:04:10.588383: step 10, loss 2.13896, acc 0.515625
2017-11-04T19:04:10.954006: step 11, loss 2.67815, acc 0.40625
2017-11-04T19:04:11.304037: step 12, loss 2.1461, acc 0.578125
2017-11-04T19:04:11.761245: step 13, loss 2.1862, acc 0.515625
2017-11-04T19:04:12.055089: step 14, loss 1.62741, acc 0.515625
2017-11-04T19:04:12.389285: step 15, loss 1.94

In [ ]:
# to run the tensorboard to visualize the summaries of the graph
#tensorboard --logdir /home/ellfae/Dropbox/WORK/PhD_Research/coding/runs/1497426429/summaries/

In [ ]:
cnn.W